In [1]:
from __future__ import print_function

# GrovePi
import grovepi# import *

# LCD
from drivers.LCD import *
# import decimal
# import drivers.grove_rgb_lcd as grove_rgb_lcd  #import *
# import drivers.grove_oled as grove_oled #import *

# Auxilary packages
import time, math, csv, os
import random as rd
import numpy as np

noiseTreshold = 700
distanceTreshold0 = 30
distanceTreshold1 = 600
lightTreshold = 100
gasTreshold = 0.7 #----------------------TODO: the sensor needs to be calibrated

# Connected devices
Devices = {
    'sound_sensor': 1,    
    'gas_sensor': 0,
    'photoresistor': 2,
    'led_red': 4,
    'led_green': 3,
    'led_yellow': 2,
    'ledbar': 5,
    'dht_sensor': 7,
    'ultrasonic_ranger': 8
}

#sentences that will appear if a sensor theshold is exceeded
sentences_sound = [
        "I've heard a noise!", 
        "Who's there?", 
        'I can hear you!',
        'Who is making noise?'
        ]
sentences_distance0 = [
        'This is ultrasonic sensor', 
        'Hello there!', 
        'Ultrasonic sensor says it is close',
        'Proximity alert!'
        ]
sentences_distance1 = [
        'I can see you!', 
        'Who is there?', 
        'Someone is in front of me.',
        'Hello?'
        ]
sentences_light = [
        'This is dark', 
        'Who turned off the light?', 
        'Did you cover photo diod?',
        'Photo diod says it is dark'
        ]
sentences_gas = [
        'I smell gas!', 
        'Gas norm exceded',
        'Combustible gas?',
        'Gas sensor is picking something'
        ]

In [2]:
unit_test = False

if unit_test:
    lcd = LCD()
    lcd.defaultColour()
    
    lcd.newMessage('hello0', colour=[10,200,200])
    lcd.newMessage('hello1', colour=[10,200,100])
    lcd.newMessage('hello2', colour=[200,100,10])

    print(lcd.pendingMessages)
    
    for i in lcd.pendingMessages:
        lcd.displayMessage()
        time.sleep(2)
        lcd.endDisplayMessage()
        
# LED Bar methods
# grovepi.ledBar_init(pin,orientation)
# grovepi.ledBar_orientation(pin,orientation)
# grovepi.ledBar_setLevel(pin,level)
# grovepi.ledBar_setLed(pin,led,state)
# grovepi.ledBar_toggleLed(pin,led)
# grovepi.ledBar_setBits(pin,state)
# grovepi.ledBar_getBits(pin)

In [3]:
computationalSpeed = []

def resetLEDs():      
    grovepi.analogWrite(Devices['led_green'],0) 
    grovepi.digitalWrite(Devices['led_yellow'],0)
    grovepi.digitalWrite(Devices['led_red'],0)    
    grovepi.ledBar_init(Devices['ledbar'], 1)
    grovepi.ledBar_setBits(Devices['ledbar'], 0)
    
def resetPINs():
    grovepi.pinMode(Devices['led_red'],"OUTPUT")
    grovepi.pinMode(Devices['led_green'],"OUTPUT")
    grovepi.pinMode(Devices['led_yellow'],"OUTPUT")
    grovepi.pinMode(Devices['ledbar'],"OUTPUT")
    grovepi.pinMode(Devices['photoresistor'],"INPUT")
    grovepi.pinMode(Devices['sound_sensor'],"INPUT")
    grovepi.pinMode(Devices['gas_sensor'],"INPUT")
    grovepi.pinMode(Devices['ultrasonic_ranger'],"INPUT")
    grovepi.pinMode(Devices['dht_sensor'],"INPUT")     
    
def rectify(param,paramTreshold):
    if param < paramTreshold:
        return 1
    else:   
        return 0 


def mainEvent(fps = 1.0, lcd_fps = 0.25, messageDuration = 3.0, readCooldown = 0.01, lcdMessages=True, interactive = True, exitOnReadError=False):       
    
    spf = 1 / fps
    lcdMain_spf = 1 / lcd_fps  
    frametime = spf    
    lcdMain_frametime = lcdMain_spf
    lcdMain_starttime = time.time()        
    lcdMessage_starttime = 3.0
    lcdMessage_frametime = 3.0   
    messageInProgress = False   
    
    lcd = LCD()
    resetPINs()
    resetLEDs()    
    dst_prev, dst_next = 0, 0
    lt_prev, lt_next = 0, 0
    lcd.defaultColour()
        
    
    outfilename = os.path.join('readings',time.strftime('%Y-%m-%d_%H:%M:%S', time.localtime(time.time())) + '.csv')
    with open(outfilename, 'w') as csvfile:
        print("file <%s> open" % outfilename)
        fieldnames = ['starttime','distance','light','sound','hum','temp','gas']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()        
        try:
            print('date_and_time        dist   light  sound  hum    temp   gas  ')
            while True:                  
                if frametime >= spf:
                    starttime = time.time()               

                    # reading from sensors analogWrite
                    try:
                        time.sleep(readCooldown)
                        sound = grovepi.analogRead(Devices['sound_sensor'])                        
                        time.sleep(readCooldown)
                        light = grovepi.analogRead(Devices['photoresistor'])
                        time.sleep(readCooldown)
                        distance = grovepi.ultrasonicRead(Devices['ultrasonic_ranger'])
                        time.sleep(readCooldown)
                        [temp, hum] = grovepi.dht(Devices['dht_sensor'], 0) 
                        time.sleep(readCooldown)
                        gas = (float)(grovepi.analogRead(Devices['gas_sensor']) / 1023)      
                        time.sleep(readCooldown)
                    except TypeError:
                        distance,light,sound,hum,temp,gas = -1,-1,-1,-1,-1,-1
                        if exitOnReadError:
                            raise SystemExit()                   

                    # write to CSV                
                    t = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(starttime))
                    if not interactive:
                        writer.writerow({'starttime':starttime,'distance':distance,'light':light,'sound':sound,'hum':hum,'temp':temp,'gas':gas})
                    
                    # message on LCD ?
                    if lcdMessages:
                        if not messageInProgress:
                            if sound > noiseTreshold:     
                                sentence = rd.randint(0,len(sentences_sound)-1)
                                lcd.newMessage(sentences_sound[sentence], colour=[10,200,200])
                            if distance < distanceTreshold0 and distance>0:
                                sentence = rd.randint(0,len(sentences_distance0)-1)
                                lcd.newMessage(sentences_distance0[sentence], colour=[10,200,10]) 
                            if light < lightTreshold:
                                sentence = rd.randint(0,len(sentences_light)-1)
                                lcd.newMessage(sentences_light[sentence], colour=[200,200,10])  
                            if gas > gasTreshold:
                                sentence = rd.randint(0,len(sentences_gas)-1)
                                lcd.newMessage(sentences_gas[sentence], colour=[250,50,10])  

                        if len(lcd.pendingMessages) > 0: 
                            if not messageInProgress:
                                lcdMessage_starttime = time.time()
                                lcd.displayMessage() 
                                messageInProgress = True

                        if messageInProgress:
                            if lcdMessage_frametime >= messageDuration:
                                lcd.endDisplayMessage()     
                                messageInProgress = False

                    if lcdMain_frametime >= lcdMain_spf:
                        if not messageInProgress:
                            lcdMain_starttime = time.time() 
                            stringForLCD = 'lumi temp  humid%4d%5.1fC%5.1f%%' % (light, temp, hum)
                            lcd.updateTextVar(stringForLCD)
                            lcd.updateText()
                    
                    if interactive:
                        #if interactive:
                        #    grovepi.ledBar_setLevel(Devices['ledbar'],int(sound/102.3))
                        #LED reaction
    #                     grovepi.analogWrite(Devices['led_green'],int(sound/4))
                        #grovepi.analogWrite(Devices['led_green'],255-int(light/4))
                        grovepi.ledBar_setLevel(Devices['ledbar'],int(sound/102.3))
                        grovepi.analogWrite(Devices['led_green'],255-int(distance*255/1023))   #int(distance/4))
                        
                        #---avoiding excessive write to LEDs                
                        dst_next = rectify(distance, distanceTreshold0)                
                        if dst_prev != dst_next:  
                            grovepi.digitalWrite(Devices['led_red'],dst_next)
                            dst_prev = dst_next     
                        
                        
                        lt_next = rectify(light,lightTreshold)
                        if lt_prev != lt_next:
                            grovepi.digitalWrite(Devices['led_yellow'],lt_next)
                            lt_prev = lt_next
                    
                    computationalSpeed.append(time.time()-starttime)
                    if time.time()-starttime > fps:
                        print('\nwarning: frame skipping')
                    #print('%s, %dcm, %dlum,%ddB' % (t,distance,light,sound) )  
                    print('%s, %dcm, %dlum,%ddB, %3.1f%%, %3.1fC, %3.3f   ' % (t,distance,light,sound,hum,temp,gas), end='\r')
                    
    #             else:
    #                 if grovepi.analogRead(Devices['sound_sensor']) > noiseTreshold:     
    #                     sentence = rd.randint(0,len(sentences_sound)-1)
    #                     lcd.newMessage(sentences_sound[sentence], colour=[10,200,200])

                # FIXED TIME STEP technique
                frametime = time.time() - starttime
                lcdMain_frametime = time.time() - lcdMain_starttime
                lcdMessage_frametime = time.time() - lcdMessage_starttime
        except (KeyboardInterrupt, SystemExit):
            resetLEDs()      
            grovepi.ledBar_setBits(Devices['ledbar'], 0)
            lcd.reset()
            print("\nprocess interrupted %s" % time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))            
            
    print("file <%s> closed" % outfilename)    
    aipt = np.mean(computationalSpeed)
    print("\n\naverage iteration processing time: %5.10f ms" % aipt)
    print("maximum fps possible: %5.10f ms" % (1/aipt))
           
mainEvent(fps=1.0, exitOnReadError=True, readCooldown = 0.005, interactive = True, lcdMessages=True)

mainEvent(fps=1.0, lcdMessages=False, exitOnReadError=True, readCooldown = 0.05)

file <readings/2017-12-12_19:56:30.csv> open
date_and_time        dist   light  sound  hum    temp   gas  
2017-12-12 19:58:11, 517cm, 591lum,290dB, 41.0%, 22.0C, 0.505               
process interrupted 2017-12-12 19:58:12
file <readings/2017-12-12_19:56:30.csv> closed


average iteration processing time: 0.4650963497 ms
maximum fps possible: 2.1500921274 ms


In [3]:
print("\nprocess interrupted %s" % time.strftime('%Y-%m-%d %H:%M:%S', time.time()))

TypeError: Tuple or struct_time argument required

In [6]:
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))

'2017-11-24 23:14:19'

In [5]:
outfilename = os.path.join('readings',time.strftime('%Y-%m-%d_%H:%M:%S', time.localtime(time.time())) + '.csv')
with open(outfilename, 'w') as csvfile:
    print("file opened <%s>" % outfilename)
print("file closed <%s>" % outfilename)

file opened <readings/2017-11-24_23:13:42.csv>
file closed <readings/2017-11-24_23:13:42.csv>


In [9]:
resetLEDs()

In [ ]:
outfilename = 'readings.csv'
with open(outfilename, 'w') as csvfile:
    print("writing to <%s>" % outfilename)
    fieldnames = ['starttime','distance','light','sound','hum','temp']

    starttime = time.time()
    sound = 55
    distance = 527    
    light = 600
    temp = 25.6
    hum = 20.2

    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerow({'starttime':starttime,'distance':distance,'light':light,'sound':sound,'hum':hum,'temp':temp})
print("<%s> closed" % outfilename)

In [9]:
unit_test = False
if unit_test:

    light = 653
    temp = 25.6
    hum = 20.2

#     stringForLCD = 'lumi temp  humid'+str(int(light*999/1024))+'  '+str(temp)+'C '+str(hum)+'%'
    stringForLCD = 'lumi temp  humid%4d%5.1fC%5.1f%%' % (light, temp, hum)
    lcd.updateTextVar(stringForLCD)
    lcd.updateText()

In [17]:
computationalSpeed = []

def event(fps=1.0, distanceTreshold0=25):    
    frametime = fps
    
    dst_prev, dst_next = 0, 0   
    lt_prev, lt_next = 0, 0
    
    try:
        while True:
            if frametime >= 1/fps:
                starttime = time.time()  
                
                # reading from sensors        
                sound = grovepi.analogRead(Devices['sound_sensor'])     
#                 sound = 0
                light = grovepi.analogRead(Devices['photoresistor'])
                distance = grovepi.ultrasonicRead(Devices['ultrasonic_ranger'])
                [temp, hum] = grovepi.dht(Devices['dht_sensor'], 0)        
#                 gas = grovepi.analogRead(Devices['gas_sensor'])  
                
                #LED reaction
#                 grovepi.analogWrite(Devices['led_green'],int(sound/4))
                #grovepi.analogWrite(Devices['led_green'],255-int(light/4))
                grovepi.ledBar_setLevel(Devices['ledbar'],int(sound/102.3))  
                #---avoiding excessive write to LEDs                
                dst_next = rectify(distance, distanceTreshold0)                
                if dst_prev != dst_next:
                    grovepi.digitalWrite(Devices['led_red'],dst_next)
                    dst_prev = dst_next                    
                
                lt_next = rectify(light,lightTreshold)
                if lt_prev != lt_next:
                    grovepi.digitalWrite(Devices['led_yellow'],lt_next)
                    lt_prev = lt_next
                    
                print('%s, %5dcm, %5dlum, %5d' % (time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(starttime)),distance, light, sound), end='\r')
                
                computationalSpeed.append(time.time()-starttime)
            # FIXED TIME STEP technique
            frametime = time.time() - starttime

    except (KeyboardInterrupt, SystemExit):
        grovepi.analogWrite(Devices['led_green'],0)  
        grovepi.digitalWrite(Devices['led_yellow'],0)
        grovepi.digitalWrite(Devices['led_red'],0)
        aipt = np.mean(computationalSpeed)
        print("\n\naverage iteration processing time: %5.10f ms" % aipt)
        print("maximum fps possible: %5.10f ms" % (1/aipt))
        if fps > 1/aipt:
            print("warning: frame skipping")

event()

2017-11-22 01:28:05,   266cm,   567lum,   320

average iteration processing time: 0.0785594881 ms
maximum fps possible: 12.7292071870 ms


In [14]:
grovepi.digitalWrite(4,1)
# grovepi.analogWrite(Devices['led_red'],100)

1

In [11]:
grovepi.ledBar_init(5, 1)
grovepi.ledBar_setLevel(5, 6)

1

In [44]:
# grovepi.analogWrite(3,200)  
# grovepi.digitalWrite(2,1)
# grovepi.digitalWrite(Devices['led_yellow'],0)
# grovepi.digitalWrite(Devices['led_red'],0)

1

In [39]:
grovepi.analogWrite(3,0)

1

In [44]:
# for i in range(1023):
#     sound = analogRead(Devices['sound_sensor'])
#     analogWrite(Devices['led_green'],i)
#     if i%10==0:
#         time.sleep(0.3)
#     print('%d, %f' % (i,sound), end='\r', flush=True)

In [5]:
# if False:
#     outfilename = 'readings.csv'
#     with open(outfilename, 'w') as csvfile:
#         print("writing to <%s>" % outfilename)
#         fieldnames = ['starttime','distance','light','sound','hum','temp']

#         starttime = time.time()
#         sound = 55
#         distance = 527    
#         light = 600
#         temp = 25.6
#         hum = 20.2

#         writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

#         writer.writeheader()
#         writer.writerow({'starttime':starttime,'distance':distance,'light':light,'sound':sound,'hum':hum,'temp':temp})
#     print("<%s> closed" % outfilename)

writing to <readings.csv>
<readings.csv> closed


In [13]:
# # from grovepi import *
# # from grove_oled import *


# #Start and initialize the OLED
# oled_init()
# oled_clearDisplay()
# oled_setNormalDisplay()
# oled_setVerticalMode()
# time.sleep(.1)

# while True:
#     try:
#         [ temp,hum ] = dht(dht_sensor,1)		#Get the temperature and Humidity from the DHT sensor
#         string = "temp ="+ str(temp)+ "C\thumidity ="+ str(hum)+"%"
#         print(string, end='\r', flush=True)
#         t = str(temp)
#         h = str(hum)

#         oled_setTextXY(0,1)			#Print "WEATHER" at line 1
#         oled_putString("WEATHER")

#         oled_setTextXY(2,0)			#Print "TEMP" and the temperature in line 3
#         oled_putString("Temp:")
#         oled_putString(t+'C')

#         oled_setTextXY(3,0)			#Print "HUM :" and the humidity in line 4
#         oled_putString("Hum :")
#         oled_putString(h+"%")
#     except (IOError,TypeError) as e:
#         print("Error", end='\r', flush=True)



In [14]:
# import decimal
# from grovepi import *
# from drivers.grove_rgb_lcd import *


# dht_sensor_port = 7     # Connect the DHt sensor to port 7
# lastTemp = 0.1          # initialize a floating point temp variable
# lastHum = 0.1           # initialize a floating Point humidity variable
# tooLow = 62.0           # Lower limit in fahrenheit
# justRight = 68.0        # Perfect Temp in fahrenheit
# tooHigh = 74.0          # Temp Too high


# # Function Definitions
# def CtoF( tempc ):
#    "This converts celcius to fahrenheit"
#    tempf = round((tempc * 1.8) + 32, 2);
#    return tempf;

# def FtoC( tempf ):
#    "This converts fahrenheit to celcius"
#    tempc = round((tempf - 32) / 1.8, 2)
#    return tempc;

# def calcColorAdj(variance):     # Calc the adjustment value of the background color
#     "Because there is 6 degrees mapping to 255 values, 42.5 is the factor for 12 degree spread"
#     factor = 42.5;
#     adj = abs(int(factor * variance));
#     if adj > 255:
#         adj = 255;
#     return adj;

# def calcBG(ftemp):
#     "This calculates the color value for the background"
#     variance = ftemp - justRight;   # Calculate the variance
#     adj = calcColorAdj(variance);   # Scale it to 8 bit int
#     bgList = [0,0,0]               # initialize the color array
#     if(variance < 0):
#         bgR = 0;                    # too cold, no red
#         bgB = adj;                  # green and blue slide equally with adj
#         bgG = 255 - adj;
        
#     elif(variance == 0):             # perfect, all on green
#         bgR = 0;
#         bgB = 0;
#         bgG = 255;
        
#     elif(variance > 0):             #too hot - no blue
#         bgB = 0;
#         bgR = adj;                  # Red and Green slide equally with Adj
#         bgG = 255 - adj;
        
#     bgList = [bgR,bgG,bgB]          #build list of color values to return
#     return bgList;

# while True:

#     try:
#         temp = 0.01
#         hum = 0.01
#         [ temp,hum ] = dht(dht_sensor_port,0)       #Get the temperature and Humidity from the DHT sensor
#                                                     #Change the second parameter to 0 when using DHT (instead of DHT Pro)
#                                                     #You will get very large number values if you don't!
#         if (CtoF(temp) != lastTemp) and (hum != lastHum) and not math.isnan(temp) and not math.isnan(hum):
#                 print("lowC : ",FtoC(tooLow),"C\t\t","rightC  : ", FtoC(justRight),"C\t\t","highC : ",FtoC(tooHigh),"C") # comment these three lines
#                 print("lowF : ",tooLow,"F\t\tjustRight : ",justRight,"F\t\ttoHigh : ",tooHigh,"F")                       # if no monitor display
#                 print("tempC : ", temp, "C\t\ttempF : ",CtoF(temp),"F\t\tHumidity =", hum,"%\r\n")
                
#                 lastHum = hum          # save temp & humidity values so that there is no update to the RGB LCD
# #                 ftemp = CtoF(temp)     # unless the value changes
#                 lastTemp = temp       # this reduces the flashing of the display
#                 # print "ftemp = ",ftemp,"  temp = ",temp   # this was just for test and debug
                
#                 bgList = calcBG(temp)           # Calculate background colors
                
#                 t = str(temp)   # "stringify" the display values
#                 h = str(hum)
#                 # print "(",bgList[0],",",bgList[1],",",bgList[2],")"   # this was to test and debug color value list
#                 setRGB(bgList[0],bgList[1],bgList[2])   # parse our list into the color settings
#                 setText("Temp:" + t + "C      " + "Humidity :" + h + "%") # update the RGB LCD display
                
#     except (IOError,TypeError) as e:
#         print("Error" + str(e))
    

